# Expirements for Task 2

According to Task 2, I will conduct the follwings expirements:
- (1) Compare Skip-gram, Skip-gram negative sampling, GloVe models on training loss, training time. 
- (2) Use Word analogies dataset to calucalte between syntactic and semantic accuracy, similar to the
methods in the Word2Vec and GloVe paper. 
- (3) Use the similarity dataset to find the correlation between your models’ dot product and the provided similarity metrics. (from scipy.stats import spearmanr) Assess if your embeddings correlate with human judgment.

## 1. Importing

In [1]:
# import 3 models from main.py
from helper import Skipgram, SkipgramNeg, Glove

In [2]:
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [3]:
import torch
import torch.nn.functional as F
import pickle
import numpy as np
from scipy.stats import spearmanr

In [4]:
# Importing training data
Data = pickle.load(open('./model/Data.pkl', 'rb'))
corpus = Data['corpus']
vocab = Data['vocab']
word2index = Data['word2index']
voc_size = Data['voc_size']
embed_size = Data['embedding_size']

## 2. Load the Model

In [5]:
# create object of the Skipgram model and load parameters
Skipgram = Skipgram(voc_size, embed_size)
Skipgram.load_state_dict(torch.load('model/A1-Skipgram.pt'))
Skipgram.eval()

Skipgram(
  (embedding_center): Embedding(4167, 2)
  (embedding_outside): Embedding(4167, 2)
)

In [ ]:
# create object of the SkipgramNeg model and load parameters
SkipgramNeg = SkipgramNeg(voc_size, embed_size)
SkipgramNeg.load_state_dict(torch.load('model/A1-NegSampling.pt'))
SkipgramNeg.eval()

In [ ]:
# Instantiate the model and load saved parameters
glove = Glove(voc_size, embed_size)
glove.load_state_dict(torch.load('model/A1-Glove.pt'))
glove.eval()

In [8]:
# you have to put this file in some python/gensim directory; just run it and it will inform where to put
glove_file = ('./glove.6B/glove.6B.100d.txt')
gensim = KeyedVectors.load_word2vec_format(glove_file, binary = False, no_header = True)

## 3. Function for semantic and syntactic analysis 
that calculates similarities using cosine similarity

In [9]:
import torch
import torch.nn.functional as F

def calculate_similarity(word_vectors, result_vector):
    
    # Calculate cosine similarities
    similarities = F.cosine_similarity(result_vector, word_vectors)

    # Find the index of the word with the highest similarity
    closest_word_index = torch.argmax(similarities).item()

    return closest_word_index

def similarities(lines, model, vocab):
    
    # Get word vectors for all words in the vocabulary
    all_word_vectors = torch.stack([model.get_embed(word) for word in vocab])
    # [voc_size, 1, emb_size]

    correct_count = 0
    # Perform vector manipulation for each set of four words
    for line in lines: # ['dancing', 'danced', 'falling', 'fell']
        words = line
        
        # Assuming there are four words in each line
        vectors = [model.get_embed(word) if word in vocab else model.get_embed('<UNK>') for word in words]
        
        # Perform vector manipulation (e.g., subtraction, addition)
        result_vector = vectors[1][0] - vectors[0][0] + vectors[2][0] # [, emb_size]

        # Add a batch dimension to result_vector
        result_vector = result_vector.unsqueeze(0) # [1, emb_size]

        # Find the closest word index using cosine similarity
        closest_word_index = calculate_similarity(all_word_vectors, result_vector)

        # Get the closest word from your vocabulary
        closest_word = vocab[closest_word_index]

        if closest_word == words[3]:
            correct_count += 1
            
    return (correct_count / len(lines)) * 100

## 4. Testing Phase

### Load the Data

For the testing data, I will use Word analogies dataset to calucalte between syntactic and semantic accuracy

In [10]:
# path to file
txt_file = '../A1 - Search Engine/word-test.v1.txt'

In [11]:
analogy = []
# current_analogy_set = {}

with open(txt_file, 'r', encoding='utf-8') as file:
    
    # Read all lines from the file and store them in a list
    lines = file.readlines()

i = 0
# Iterate through lines and extract analogy sets
for line in lines:
    line = line.strip() # remove the space
    
    # check if the line is section declaration line < : capital-common-country >
    # if yes >> take as section title
    if line.startswith(':'):
        
        if i > 0:
            analogy.append(current_analogy_set)
        
        current_section = line[2:]
        current_analogy_set = {'section': current_section, 'pairs': []}
    
    # otherwise >> record as each word and append into a list called pairs for each section
    else:
        words = line.split()
        current_analogy_set['pairs'].append(words)
    
    i+=1
        
    # if not line:  # Empty line indicates the end of an analogy set
    #     analogy.append(current_analogy_set)

# Display the first analogy set for illustration
print(analogy[0])


{'section': 'capital-common-countries', 'pairs': [['Athens', 'Greece', 'Baghdad', 'Iraq'], ['Athens', 'Greece', 'Bangkok', 'Thailand'], ['Athens', 'Greece', 'Beijing', 'China'], ['Athens', 'Greece', 'Berlin', 'Germany'], ['Athens', 'Greece', 'Bern', 'Switzerland'], ['Athens', 'Greece', 'Cairo', 'Egypt'], ['Athens', 'Greece', 'Canberra', 'Australia'], ['Athens', 'Greece', 'Hanoi', 'Vietnam'], ['Athens', 'Greece', 'Havana', 'Cuba'], ['Athens', 'Greece', 'Helsinki', 'Finland'], ['Athens', 'Greece', 'Islamabad', 'Pakistan'], ['Athens', 'Greece', 'Kabul', 'Afghanistan'], ['Athens', 'Greece', 'London', 'England'], ['Athens', 'Greece', 'Madrid', 'Spain'], ['Athens', 'Greece', 'Moscow', 'Russia'], ['Athens', 'Greece', 'Oslo', 'Norway'], ['Athens', 'Greece', 'Ottawa', 'Canada'], ['Athens', 'Greece', 'Paris', 'France'], ['Athens', 'Greece', 'Rome', 'Italy'], ['Athens', 'Greece', 'Stockholm', 'Sweden'], ['Athens', 'Greece', 'Tehran', 'Iran'], ['Athens', 'Greece', 'Tokyo', 'Japan'], ['Baghdad', 'I

In [12]:
# get the corpus_capital_country = where section = 'capital country'
for j in range(len(analogy)):
    if analogy[j]['section'] == 'capital-common-countries':
        corpus_capital_country = analogy[j]['pairs']
    
    if analogy[j]['section'] == 'gram7-past-tense':
        corpus_past_tense = analogy[j]['pairs']
        
# corpus_capital_country
# corpus_past_tense

### (1) syntactic and semantic accuracy

#### Skipgram Model

In [13]:
# Sementic
accuracy = similarities (corpus_capital_country, Skipgram, vocab)

print(f"Accuracy : {accuracy:.2f} %")

Accuracy : 0.00 %


In [14]:
# Syntactic
accuracy = similarities (corpus_past_tense, Skipgram, vocab)

print(f"Accuracy : {accuracy:.2f}")

Accuracy : 0.00


#### Skipgram (Neg) Model

In [15]:
# Sementic
accuracy = similarities (corpus_capital_country, SkipgramNeg, vocab)

print(f"Accuracy : {accuracy:.2f}")

Accuracy : 0.00


In [16]:
# Syntactic
accuracy = similarities (corpus_past_tense, SkipgramNeg, vocab)

print(f"Accuracy : {accuracy:.2f}")

Accuracy : 0.00


#### GloVe Model

In [17]:
# Sementic
accuracy = similarities (corpus_capital_country, glove, vocab)

print(f"Accuracy : {accuracy:.2f}")

Accuracy : 0.00


In [18]:
# Syntactic
accuracy = similarities (corpus_past_tense, glove, vocab)

print(f"Accuracy : {accuracy:.2f}")

Accuracy : 0.00


#### Gensim Model

In [19]:
# analogy
def analogy(x1, x2, y1):
    result = gensim.most_similar(positive=[y1,x2], negative = [x1])
    return result[0][0]

In [20]:
def gensim_similarity (lines):
    
    correct_count = 0
    
    for line in lines:
        words = line # ['dancing', 'danced', 'falling', 'fell']
        
        # check whether words are in gensim model or not
        words_checked = [word.lower() if word.lower() in gensim else 'unknown' for word in words] 
        # print (words_checked)
        
        # result = gensim.most_similar(positive = [words_checked[2], words_checked[1]], negative = [words_checked[0]])
        # word : number (queen : 0.7699)
        result = analogy (words_checked[0], words_checked[1], words_checked[2])
        
        if result == words_checked[3]:
            correct_count += 1  
        
    return (correct_count / len(lines)) * 100

In [21]:
# sementic
accuracy = gensim_similarity (corpus_capital_country)

print(f"Accuracy : {accuracy:.2f}")

Accuracy : 93.87


In [22]:
# syntatic
accuracy = gensim_similarity (corpus_past_tense)

print(f"Accuracy : {accuracy:.2f}")

Accuracy : 55.45


### (2) Use Similarity dataset and find the Correlation

In [23]:
# load the txt file
wordsim_filepath = '../A1 - Search Engine/wordsim_similarity_goldstandard.txt'

with open(wordsim_filepath, 'r') as file:
    wordsim = file.readlines()
    print(wordsim)

['tiger\tcat\t7.35\n', 'tiger\ttiger\t10.00\n', 'plane\tcar\t5.77\n', 'train\tcar\t6.31\n', 'television\tradio\t6.77\n', 'media\tradio\t7.42\n', 'bread\tbutter\t6.19\n', 'cucumber\tpotato\t5.92\n', 'doctor\tnurse\t7.00\n', 'professor\tdoctor\t6.62\n', 'student\tprofessor\t6.81\n', 'smart\tstupid\t5.81\n', 'wood\tforest\t7.73\n', 'money\tcash\t9.15\n', 'king\tqueen\t8.58\n', 'king\trook\t5.92\n', 'bishop\trabbi\t6.69\n', 'fuck\tsex\t9.44\n', 'football\tsoccer\t9.03\n', 'football\tbasketball\t6.81\n', 'football\ttennis\t6.63\n', 'Arafat\tJackson\t2.50\n', 'physics\tchemistry\t7.35\n', 'vodka\tgin\t8.46\n', 'vodka\tbrandy\t8.13\n', 'drink\teat\t6.87\n', 'car\tautomobile\t8.94\n', 'gem\tjewel\t8.96\n', 'journey\tvoyage\t9.29\n', 'boy\tlad\t8.83\n', 'coast\tshore\t9.10\n', 'asylum\tmadhouse\t8.87\n', 'magician\twizard\t9.02\n', 'midday\tnoon\t9.29\n', 'furnace\tstove\t8.79\n', 'food\tfruit\t7.52\n', 'bird\tcock\t7.10\n', 'bird\tcrane\t7.38\n', 'food\trooster\t4.42\n', 'money\tdollar\t8.42\n

In [24]:
# cosine similarity
def cosine_similarity (A,B):
    dot_product = np.dot(A.flatten(),B.flatten())
    norm_a      = np.linalg.norm(A)
    nomr_b      = np.linalg.norm(B)
    similarity  = dot_product / (norm_a * nomr_b)
    return similarity

In [25]:
# calulate correlation

def correlation_func (txt, model, vocab):
    predicted_scores = []
    actual_scores    = []
    # cos_sim_value = 0
    
    for line in txt:
        
        # Clean and tokenize the line
        line = line.strip().split() # ['cat','tiger','7.3']
        
        # Get embeddings for each word in the line, or use '<UNK>' if not in vocab
        word_checked = [model.get_embed(word) if word in vocab else model.get_embed('<UNK>') for word in line[:-1]]
        # print(word_checked[0])
        
        # cal cosine similarity
        cosine_sim_value = cosine_similarity(word_checked[0].detach().numpy(), word_checked[1].detach().numpy())
        
        predicted_scores.append(cosine_sim_value)
        actual_scores.append(float(line[-1]))
        
        result = spearmanr(predicted_scores, actual_scores)
        
    return result[0]

In [37]:
# to store all the model and correlation result
correlation_result = {}

#### 1. Skipgram model

In [38]:
print(f"Skipgram correlation score: {correlation_func(wordsim, Skipgram, vocab)}")
skipgramCorrelation = correlation_func(wordsim, Skipgram, vocab)
correlation_result['skipgram'] = skipgramCorrelation

Skipgram correlation score: 0.09390973994656401


#### 2. Skipgram (Negative Sampling) model

In [39]:
print(f"NegSkipgram correlation score: {correlation_func(wordsim, SkipgramNeg, vocab)}")
negCorrelation = correlation_func(wordsim, SkipgramNeg, vocab)
correlation_result['Neg'] = negCorrelation

NegSkipgram correlation score: 0.08443987589946488


#### 3. GloVe model

In [40]:
print(f"GloVe correlation score: {correlation_func(wordsim, glove, vocab)}")
gloveCorrelation = correlation_func(wordsim, glove, vocab)
correlation_result['glove'] = gloveCorrelation

GloVe correlation score: 0.0844819200165613


#### 4. Gensim  model

In [41]:
def gensim_dist(txt):

    gen_dist = []
    actual_dist = []

    for line in txt:
        
        # Clean and tokenize the line
            line = line.strip().split() # ['cat','tiger','7.3']
            
            # Get embeddings for each word in the line, or use '<UNK>' if not in vocab
            word_checked = [word.lower() if word.lower() in gensim else 'unknown' for word in line[:-1]]
            
            dist = gensim.distance(word_checked[0], word_checked[1])
            
            gen_dist.append(dist)
            actual_dist.append(float(line[-1]))
            
    return spearmanr(gen_dist, actual_dist)[0]

In [42]:
gensimCorrelation = gensim_dist(wordsim)
print(f"{gensimCorrelation}")
correlation_result['Gensim'] = gensimCorrelation

-0.6035208452876207


In [43]:
for model, correlationS in correlation_result.items():
    print(f"{model} correlation: {correlationS}")

skipgram correlation: 0.09390973994656401
Neg correlation: 0.08443987589946488
glove correlation: 0.0844819200165613
Gensim correlation: -0.6035208452876207


The Similarity Metrics (Correlation between my models and human judgement)

Model             | Skipgram | NEG | GloVe | GloVe (gensim) | Y-true |
|----------------- |---------|-----|-------|----------------|--------|
| Spearmanr Result | 0.0939  | 0.0844 | 0.0845  | -0.6035   |   |